<a href="https://colab.research.google.com/github/twister0316/Public/blob/main/excel_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import os

# ---------------------------------------------------------
# [진단] 현재 코랩에 업로드된 파일 목록을 출력합니다.
# ---------------------------------------------------------
print("📂 현재 업로드된 파일 목록:")
files = os.listdir()
print(files)
print("-" * 30)

# ---------------------------------------------------------
# [설정] 업로드한 파일 이름이 정확해야 합니다.
# ---------------------------------------------------------
file_krx = "상장사현황_KRX.xlsx - Sheet1.csv"
file_info = "상장기업_전체정보_2023.xlsx - Sheet1.csv"

# 파일이 제대로 업로드되었는지 확인
#if not os.path.exists(file_krx) or not os.path.exists(file_info):
#    print("❌ [오류] 파일을 찾을 수 없습니다!")
#    print("👉 왼쪽 폴더 아이콘(📂)을 누르고 파일을 드래그해서 업로드해주세요.")
#    print(f"   (필요한 파일명: {file_krx}, {file_info})")

# 파일 존재 여부 체크
if not os.path.exists(file_krx):
    print(f"❌ 오류: '{file_krx}' 파일을 찾을 수 없습니다.")
    print("👉 해결책: 왼쪽 폴더 아이콘을 눌러 파일을 업로드하거나, 파일명을 코드와 똑같이 수정하세요.")
elif not os.path.exists(file_info):
    print(f"❌ 오류: '{file_info}' 파일을 찾을 수 없습니다.")
    print("👉 해결책: 왼쪽 폴더 아이콘을 눌러 파일을 업로드하거나, 파일명을 코드와 똑같이 수정하세요.")

else:
    print("✅ 파일 확인 완료! 통합 작업을 시작합니다...")

    # 1. 파일 읽기 (인코딩 에러 방지)
    # 한글 엑셀 파일은 utf-8이 아니라 cp949로 저장된 경우가 많습니다.
    def read_csv_safe(path):
        try:
            return pd.read_csv(path, encoding='utf-8')
        except UnicodeDecodeError:
            return pd.read_csv(path, encoding='cp949')

    df_krx = read_csv_safe(file_krx)
    df_info = read_csv_safe(file_info)

    # 2. 종목코드 6자리로 통일 (가장 중요한 키 값)
    # 005930이 5930.0 처럼 숫자로 깨져있을 경우를 대비해 0을 채웁니다.
    def clean_code(x):
        try:
            # 소수점 제거 후 문자열 변환 및 0 채우기
            return str(int(float(x))).zfill(6)
        except:
            return str(x).zfill(6)

    # 컬럼명 공백 제거 (에러 방지)
    df_krx.columns = [c.strip() for c in df_krx.columns]
    df_info.columns = [c.strip() for c in df_info.columns]

    # 키 컬럼 생성
    col_krx = '단축코드' if '단축코드' in df_krx.columns else '종목코드'
    col_info = '종목코드'

    df_krx['키_종목코드'] = df_krx[col_krx].apply(clean_code)
    df_info['키_종목코드'] = df_info[col_info].apply(clean_code)

    # 3. 데이터 합치기 (Outer Join: 합집합)
    # 한쪽에만 있는 기업도 모두 포함시킵니다.
    merged_df = pd.merge(df_krx, df_info, on='키_종목코드', how='outer')

    # 4. 데이터 정리 (비어있는 정보 채우기)
    # KRX 파일의 '한글 종목명'이 비어있으면 Info 파일의 '회사명'을 가져옵니다.
    merged_df['최종_회사명'] = merged_df['한글 종목명'].fillna(merged_df['회사명'])

    # 5. 결과 파일에 남길 컬럼 선택
    final_columns = [
        '키_종목코드',      # 종목코드
        '최종_회사명',      # 회사명
        '시장구분',       # (KRX)
        '소속부',         # (KRX)
        '상장일',         # (KRX)
        '상장주식수',     # (KRX)
        '매출액(원)',     # (Info)
        '대표자',         # (Info)
        '주소',           # (Info)
        '홈페이지',       # (Info)
        '전화번호',       # (Info)
        '설립일'          # (Info)
    ]

    # 존재하는 컬럼만 선택해서 저장 (에러 방지)
    available_cols = [c for c in final_columns if c in merged_df.columns]
    result_df = merged_df[available_cols]

    # 보기 좋게 이름 변경
    result_df.rename(columns={'키_종목코드': '종목코드', '최종_회사명': '회사명'}, inplace=True)

    # 6. 엑셀로 저장
    output_filename = "상장사_통합정보_완료.xlsx"
    result_df.to_excel(output_filename, index=False)

    print("\n" + "="*50)
    print(f"🎉 통합 완료! 총 {len(result_df)}개 기업이 정리되었습니다.")
    print(f"📂 왼쪽 파일 탭에서 '{output_filename}' 파일을 우클릭하여 다운로드하세요.")
    print("="*50)

📂 현재 업로드된 파일 목록:
['.config', '상장사현황_KRX.xlsx', '.ipynb_checkpoints', '상장기업_전체정보_2023.xlsx', 'sample_data']
------------------------------
❌ 오류: '상장사현황_KRX.xlsx - Sheet1.csv' 파일을 찾을 수 없습니다.
👉 해결책: 왼쪽 폴더 아이콘을 눌러 파일을 업로드하거나, 파일명을 코드와 똑같이 수정하세요.


In [5]:
import pandas as pd
import os

# ---------------------------------------------------------
# [진단] 현재 코랩에 업로드된 파일 목록을 출력합니다.
# ---------------------------------------------------------
print("📂 현재 업로드된 파일 목록:")
files = os.listdir()
print(files)
print("-" * 30)

# ---------------------------------------------------------
# [설정] 실제 파일명을 확인 후 아래 변수에 정확히 넣어주세요.
# (파일 목록을 보고 복사해서 붙여넣으세요)
# ---------------------------------------------------------
# 예시: 만약 목록에 'KRX.csv'라고 떠있다면 아래를 "KRX.csv"로 고치세요.
file_krx = "상장사현황_KRX.xlsx - Sheet1.csv"
file_info = "상장기업_전체정보_2023.xlsx - Sheet1.csv"

# 파일 존재 여부 체크
if not os.path.exists(file_krx):
    print(f"❌ 오류: '{file_krx}' 파일을 찾을 수 없습니다.")
    print("👉 해결책: 왼쪽 폴더 아이콘을 눌러 파일을 업로드하거나, 파일명을 코드와 똑같이 수정하세요.")
elif not os.path.exists(file_info):
    print(f"❌ 오류: '{file_info}' 파일을 찾을 수 없습니다.")
    print("👉 해결책: 왼쪽 폴더 아이콘을 눌러 파일을 업로드하거나, 파일명을 코드와 똑같이 수정하세요.")

else:
    # ---------------------------------------------------------
    # [실행] 파일이 확인되면 통합 작업을 시작합니다.
    # ---------------------------------------------------------
    print("✅ 파일 확인 완료! 통합을 시작합니다...")

    try:
        df_krx = pd.read_csv(file_krx)
        df_info = pd.read_csv(file_info)
    except UnicodeDecodeError:
        df_krx = pd.read_csv(file_krx, encoding='cp949')
        df_info = pd.read_csv(file_info, encoding='cp949')

    # 종목코드 6자리 통일 (zfill)
    def clean_code(x):
        try:
            return str(int(float(x))).zfill(6)
        except:
            return str(x).zfill(6)

    # 컬럼 공백 제거 및 키 생성
    df_krx.columns = [c.strip() for c in df_krx.columns]
    df_info.columns = [c.strip() for c in df_info.columns]

    # 컬럼명 확인 후 키 매핑
    col_krx = '단축코드' if '단축코드' in df_krx.columns else '종목코드' # 유연하게 대처
    col_info = '종목코드'

    df_krx['키_종목코드'] = df_krx[col_krx].apply(clean_code)
    df_info['키_종목코드'] = df_info[col_info].apply(clean_code)

    # 병합
    merged_df = pd.merge(df_krx, df_info, on='키_종목코드', how='left')

    # 결과 정리
    final_columns = [
        '키_종목코드', '한글 종목명', '시장구분', '소속부', '상장일', '상장주식수',
        '매출액(원)', '대표자', '주소', '홈페이지', '전화번호', '설립일'
    ]

    # 존재하는 컬럼만 선택
    available_cols = [c for c in final_columns if c in merged_df.columns]
    result_df = merged_df[available_cols]
    result_df.rename(columns={'키_종목코드': '종목코드', '한글 종목명': '회사명'}, inplace=True)

    # 저장
    output_name = "상장사_통합정보_완료.xlsx"
    result_df.to_excel(output_name, index=False)

    print(f"🎉 성공! '{output_name}' 파일이 생성되었습니다. 왼쪽 폴더에서 다운로드하세요.")

📂 현재 업로드된 파일 목록:
['.config', '상장사현황_KRX.xlsx', '.ipynb_checkpoints', '상장기업_전체정보_2023.xlsx', 'sample_data']
------------------------------
❌ 오류: '상장사현황_KRX.xlsx - Sheet1.csv' 파일을 찾을 수 없습니다.
👉 해결책: 왼쪽 폴더 아이콘을 눌러 파일을 업로드하거나, 파일명을 코드와 똑같이 수정하세요.
